In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

## Below we define our custom transformers to use on pipelines

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, MinMaxScaler, OneHotEncoder


class Encoder(BaseEstimator, TransformerMixin):

    def __init__(self, cat_attributes=None, encoder_arg='one_hot'):
        self.encoder_arg = encoder_arg
        self.cat_attributes = cat_attributes

    def fit(self, X, y=None):
        cat_cols = X[self.cat_attributes]
        if self.encoder_arg == 'one_hot':
            self.encoder = OneHotEncoder(sparse=False)
        self.encoder.fit(cat_cols)
        return self

    def transform(self, X, y=None):
        encoded_df = pd.DataFrame(self.encoder.transform(X[self.cat_attributes]))
        df = pd.concat([X.drop(self.cat_attributes, axis=1), encoded_df], axis=1)
        df.columns = df.columns.astype(str)
        return df


class Imputer(BaseEstimator, TransformerMixin):

    def __init__(self, fill_with=0):
        self.fill_with = fill_with
        self.imputer = None

    def fit(self, X, y=None):
        self.imputer = self.fill_with
        return self

    def transform(self, X, y=None):
        df = X.copy().reset_index(drop=True)
        #df = X.copy()
        if self.imputer == 'median':
            return df.fillna(df.median())
        else:
            return df.fillna(self.imputer)

    def get_params(self, deep=True):
        return {'fill_with': self.fill_with}


class PolyTransformer(BaseEstimator, TransformerMixin):

    def __init__(self, degree=2, cat_attributes=[], no_higher_powers=False, \
                 rename=False, include_bias=True):
        self.degree = degree
        self.cat_attributes = cat_attributes
        self.no_higher_powers= no_higher_powers
        self.feature_names_in_ = None
        self.rename = rename
        self.include_bias=include_bias

    def fit(self, X, y=None):
        self.transformer = PolynomialFeatures(degree=self.degree, \
                interaction_only=self.no_higher_powers, include_bias=self.include_bias)
        self.df_num_cols = list(set(X.columns)-set(self.cat_attributes))
        df_num = X[self.df_num_cols]
        self.transformer.fit(df_num)
        return self

    def transform(self, X, y=None):
        df_num = X[self.df_num_cols]
        df_num_squared = pd.DataFrame(self.transformer.transform(df_num))
        if self.rename == True:
            old_columns = df_num_squared.columns
            new_columns = self.transformer.get_feature_names_out(df_num.columns)
            column_mapping = {old_col: new_col for old_col, new_col in zip(old_columns, new_columns)}
            df_num_squared = df_num_squared.rename(columns= column_mapping)
            self.squared_cols = df_num_squared.columns
        df_cat = X[self.cat_attributes]
        df_combined = pd.DataFrame(df_num_squared).join(pd.DataFrame(df_cat))
        return df_combined

    def get_feature_names(self):
        return self.transformer.get_feature_names_out(input_features=self.df_num_cols)

    def get_coefs_df(self, estimator):
        coefs_df = pd.DataFrame(0, index=self.df_num_cols, columns=self.df_num_cols)
        coefs_df.loc['1'] = 0
        coefs_df['1'] = 0
        linear_features = coefs_df.columns
        coefs = estimator.coef_
        len_classes = len(self.squared_cols)
        coefs = coefs[:len_classes]
        coef_feature = pd.DataFrame({'Feature':self.squared_cols, 'Coefficient':coefs})\
                .sort_values(by='Coefficient', key=abs, ascending=False)

        for index, row in coef_feature.iterrows():
            for column_original_1 in linear_features:
                for column_original_2 in linear_features:
                    if (column_original_1 in row['Feature']) and (column_original_2 in row['Feature'])\
                    and column_original_1 != column_original_2:
                        coefs_df.loc[column_original_1, column_original_2] = row['Coefficient']
                        coefs_df.loc[column_original_2, column_original_1] = row['Coefficient']

                    if (column_original_1 == column_original_2) and (column_original_2 in row['Feature'])\
                    and ('^' in row['Feature']):
                        coefs_df.loc[column_original_1, column_original_1] = row['Coefficient']

                    if (column_original_1 == '1') and column_original_1 != column_original_2\
                    and (column_original_2 == row['Feature']):
                        coefs_df.loc[column_original_1, column_original_2] = row['Coefficient']
                        coefs_df.loc[column_original_2, column_original_1] = row['Coefficient']

                    if (column_original_2 == '1') and column_original_1 != column_original_2\
                    and (column_original_1 == row['Feature']):
                        coefs_df.loc[column_original_1, column_original_2] = row['Coefficient']
                        coefs_df.loc[column_original_2, column_original_1] = row['Coefficient']

        # Note that the 1-1 cell is the bias term!!!

                    if (column_original_2 == '1') and (column_original_1 == '1')\
                    and (column_original_1 == row['Feature']):
                        coefs_df.loc[column_original_1, column_original_1] = row['Coefficient']

        return coefs_df


class Scaler(BaseEstimator, TransformerMixin):

    def __init__(self, scaler_type='standard', output_type=None):
        self.scaler_type = scaler_type
        self.output_type=output_type
        if output_type=='dataframe':
            self.output_type='pandas'

    def fit(self, X, y=None):
        if self.scaler_type == 'standard':
            self.transformer = StandardScaler().set_output(transform=self.output_type)
        elif self.scaler_type == 'minmax':
            self.transformer = MinMaxScaler().set_output(transform=self.output_type)
        self.transformer.fit(X)
        return self

    def transform(self, X, y=None):
        df = self.transformer.transform(X)
        return df


class OneCycleScheduler(keras.callbacks.Callback):
    def __init__(self, iterations, max_rate, start_rate=None,
                 last_iterations=None, last_rate=None):
        self.iterations = iterations
        self.max_rate = max_rate
        self.start_rate = start_rate or max_rate / 10
        self.last_iterations = last_iterations or iterations // 10 + 1
        self.half_iteration = (iterations - self.last_iterations) // 2
        self.last_rate = last_rate or self.start_rate / 1000
        self.iteration = 0
    def _interpolate(self, iter1, iter2, rate1, rate2):
        return ((rate2 - rate1) * (self.iteration - iter1)
                / (iter2 - iter1) + rate1)
    def on_batch_begin(self, batch, logs):
        if self.iteration < self.half_iteration:
            rate = self._interpolate(0, self.half_iteration, self.start_rate, self.max_rate)
        elif self.iteration < 2 * self.half_iteration:
            rate = self._interpolate(self.half_iteration, 2 * self.half_iteration,
                                     self.max_rate, self.start_rate)
        else:
            rate = self._interpolate(2 * self.half_iteration, self.iterations,
                                     self.start_rate, self.last_rate)
        self.iteration += 1
        keras.backend.set_value(self.model.optimizer.learning_rate, rate)



class OneCycleScheduler(keras.callbacks.Callback):
    def __init__(self, max_n_batches, max_rate, start_rate=None,
                 fraction_for_last_drop=0.1, final_rate=None,
                 max_momentum=0.95, min_momentum=None):
        self.max_n_batches = max_n_batches
        self.max_rate = max_rate
        self.start_rate = start_rate or max_rate / 10
        self.n_batches_for_last_drop = round(fraction_for_last_drop*max_n_batches)
        self.half_point_it = round((1-fraction_for_last_drop)*max_n_batches) // 2
        self.final_rate = final_rate or self.start_rate / 1000
        self.max_momentum = max_momentum
        self.min_momentum = min_momentum or (self.max_momentum - (self.max_momentum / 10))
        self.iteration = 0
    def _interpolate(self, iter1, iter2, rate1, rate2):
        return ((rate2 - rate1) * (self.iteration - iter1)
                / (iter2 - iter1) + rate1)
    def on_batch_begin(self, batch, logs):
        if self.iteration < self.half_point_it:
            rate = self._interpolate(0, self.half_point_it, self.start_rate, self.max_rate)
            new_momentum = self._interpolate(0, self.half_point_it, self.max_momentum, self.min_momentum)
        elif self.iteration < 2 * self.half_point_it:
            rate = self._interpolate(self.half_point_it, 2 * self.half_point_it,
                                     self.max_rate, self.start_rate)
            new_momentum = self._interpolate(self.half_point_it, 2 * self.half_point_it,
                                     self.min_momentum, self.max_momentum)
        else:
            rate = self._interpolate(2 * self.half_point_it, self.max_n_batches,
                                     self.start_rate, self.final_rate)
            # on the last steps there is no need to update momentum (it should remain constant)
            new_momentum = self.max_momentum

        self.iteration += 1

        keras.backend.set_value(self.model.optimizer.learning_rate, rate)

        if hasattr(self.model.optimizer, 'momentum'):
          new_momentum = tf.Variable(new_momentum)
          self.model.optimizer.momentum = tf.Variable(self.model.optimizer.momentum)
          keras.backend.set_value(self.model.optimizer.momentum, new_momentum)

## Load the stored preprocessed dataframe

In [ ]:
X = pd.read_json('/content/drive/MyDrive/Deep_Learning/preprocessed_elections_data_X.json')
y = pd.read_json('/content/drive/MyDrive/Deep_Learning/preprocessed_elections_data_y.json')

## Transforming the datasets

In [ ]:
imputing = Imputer(fill_with='median')
encoder = Encoder(cat_attributes=['SG_UF'])
std_scaler = StandardScaler()

X = imputing.fit_transform(X)
X = encoder.fit_transform(X)

X.isna().sum()

X = std_scaler.fit_transform(X)

<ipython-input-39-6f9feb0a73ff>:39: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  return df.fillna(df.median())
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
X.shape, y.shape

((5553, 81), (5553, 1))

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

## Defining the model

In [ ]:
# Let us build a function that creates a model specific for the 2022 Elections analysis

def build_model(n_hidden=2, n_neurons=30, activation_list=["relu", "relu", "logistic"], initialization_list=["he_normal", "he_normal"],
                learning_rate=3e-3, input_shape=[10,], loss="mse", delta=1, metrics=None,
                optimizer='sgd', momentum=0.9, beta_1=0.9, beta_2=0.999, **kwargs):
  model = keras.Sequential()
  # Instead of adding the InputLayer, could simply put the `input_shape` argument in the next layer
  model.add(keras.layers.InputLayer(input_shape=input_shape))
  for layer in range(n_hidden):
    model.add(keras.layers.Dense(n_neurons, activation=activation_list[layer], kernel_initializer=initialization_list[layer]))
  model.add(keras.layers.Dense(1, activation=activation_list[-1]))
  if optimizer=='sgd':
    optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
  elif optimizer=='momentum':
    optimizer = keras.optimizers.SGD(learning_rate=learning_rate, momentum=momentum)
  elif optimizer=='nesterov':
    optimizer = keras.optimizers.SGD(learning_rate=learning_rate, momentum=momentum, nesterov=True)
  elif optimizer=='adam':
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate, beta_1=beta_1, beta_2=beta_2)
  if loss=='huber':
    loss = tf.keras.losses.Huber(delta=delta)
  model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
  return model

In [ ]:
root_dir = "/content/drive/MyDrive/Deep_Learning/"

import os
root_logdir = os.path.join(root_dir, "my_logs")

def get_run_logdir():
  import time
  run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
  return os.path.join(root_logdir, run_id)

In [ ]:
run_logdir = get_run_logdir() # e.g., './my_logs/run_2019_06_07-15_15_22'

tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
checkpoint_cb = keras.callbacks.ModelCheckpoint("Elections_2022.h5", save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=8, restore_best_weights=True)

#history = model.fit(X_train, y_train, epochs=n_epochs, callbacks=[checkpoint_cb, early_stopping_cb, tensorboard_cb], validation_split=0.2)


In [ ]:
model.evaluate(X_test, y_test)

35/35 [==============================] - 0s 4ms/step - loss: 0.0054


0.0053786602802574635

In [ ]:
#%reload_ext tensorboard
#%tensorboard --logdir=/content/drive/MyDrive/Deep_Learning/my_logs --port=6006

In [ ]:
#!lsof -i:6006

COMMAND     PID USER   FD   TYPE DEVICE SIZE/OFF NODE NAME
tensorboa 16946 root    7u  IPv4 431743      0t0  TCP localhost:6006 (LISTEN)


In [ ]:
#!kill -9 16946

## Test 1

In [ ]:
from hyperopt import hp, fmin, tpe, Trials
from sklearn.metrics import mean_squared_error
import math

# Define the search space for hyperparameters

[
    {
     'activation_hidden': ['choice', ['selu', 'elu']],
     'initialization_hidden': ['choice', ['lecun_normal', 'he_normal']],
     'n_neurons': ['quniform', [140, 300, 40]],
     'optimizer': ['choice', ['momentum', 'nesterov', 'adam']],
     'start_learning_rate': ['loguniform', [math.log(1e-4), math.log(0.1)]],
     'max_learning_rate': ['loguniform', [math.log(0.1), math.log(10)]],

     }

 ]




n_layers_max = 5
space = hp.choice('layer_architecture',[
    hp.choice('loss_choice_{}_layers'.format(n_layers),[
  {'n_hidden': n_layers,
  'activation_list': [hp.choice('activation_hidden_mse_{}_layers'.format(n_layers), ['selu', 'elu'])]*n_layers + ['sigmoid'],
  'initialization_list': [hp.choice('initialization_hidden_mse_{}_layers'.format(n_layers), ['lecun_normal', 'he_normal'])]*n_layers,
  'n_neurons': hp.quniform('n_neurons_mse_{}_layers'.format(n_layers), 140, 300, 40),
  #'learning_rate': hp.loguniform('learning_rate_mse_{}_layers'.format(n_layers), -4, -2),
  'optimizer': hp.choice('optimizer_mse_{}_layers'.format(n_layers), ['momentum', 'nesterov', 'adam']),
  'start_learning_rate': hp.loguniform('start_lr_mse_{}_layers'.format(n_layers), math.log(1e-4), math.log(0.1)),
  'max_learning_rate': hp.loguniform('max_lr_mse_{}_layers'.format(n_layers), math.log(0.1), math.log(10)),
  'batch_size_0': hp.quniform('batch_size_mse_{}_layers'.format(n_layers), 32, 832, 50),
  'loss': 'mse'
  },
  {'n_hidden': n_layers,
  'activation_list': [hp.choice('activation_hidden_huber_{}_layers'.format(n_layers), ['selu', 'elu'])]*n_layers + ['sigmoid'],
  'initialization_list': [hp.choice('initialization_hidden_huber_{}_layers'.format(n_layers), ['lecun_normal', 'he_normal'])]*n_layers,
  'n_neurons': hp.quniform('n_neurons_huber_{}_layers'.format(n_layers), 140, 300, 40),
  #'learning_rate': hp.loguniform('learning_rate_huber_{}_layers'.format(n_layers), -4, -2),
  'optimizer': hp.choice('optimizer_huber_{}_layers'.format(n_layers), ['momentum', 'nesterov', 'adam']),
  'start_learning_rate': hp.loguniform('start_lr_huber_{}_layers'.format(n_layers), math.log(1e-4), math.log(0.1)),
  'max_learning_rate': hp.loguniform('max_lr_huber_{}_layers'.format(n_layers), math.log(0.1), math.log(10)),
  'batch_size_0': hp.quniform('batch_size_huber_{}_layers'.format(n_layers), 32, 832, 50),
  'loss': 'huber',
  'delta': hp.uniform('delta_{}_layers'.format(n_layers), 0.04, 0.1)
  }]) for n_layers in range(1, n_layers_max)])


# Objective function to minimize (or maximize)
def objective(params):
    N_EPOCHS = 100
    batch_size = int(params['batch_size_0'])
    model = build_model(**params, input_shape=(81,))

    # Assuming you have your data ready, otherwise replace X_train, y_train with your data
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

    max_learning_rate = params['max_learning_rate']
    start_learning_rate = params['start_learning_rate']


    onecycle_cb = OneCycleScheduler(N_EPOCHS*(len(X_train)//batch_size), max_learning_rate, start_rate=start_learning_rate)
    model.fit(X_train, y_train, epochs=N_EPOCHS, validation_split = 0.15, verbose=0, callbacks=[early_stopping_cb, onecycle_cb], batch_size=batch_size)

    # Evaluate model on validation data
    y_pred = model.predict(X_val)
    error = mean_squared_error(y_val, y_pred)
    return error


# Run optimization
trials = Trials()
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=100, trials=trials)

print("Best parameters:", best)


35/35 [==============================] - 0s 2ms/step

35/35 [==============================] - 0s 4ms/step

35/35 [==============================] - 0s 4ms/step

35/35 [==============================] - 0s 2ms/step

35/35 [==============================] - 0s 2ms/step

35/35 [==============================] - 0s 2ms/step

35/35 [==============================] - 0s 2ms/step

35/35 [==============================] - 0s 3ms/step

35/35 [==============================] - 0s 3ms/step

35/35 [==============================] - 0s 4ms/step

35/35 [==============================] - 0s 2ms/step

35/35 [==============================] - 0s 4ms/step

35/35 [==============================] - 0s 2ms/step

35/35 [==============================] - 0s 3ms/step

35/35 [==============================] - 0s 3ms/step

35/35 [==============================] - 0s 2ms/step

35/35 [==============================] - 0s 2ms/step

35/35 [==============================] - 0s 2ms/step

35/35 [=====================

35/35 [==============================] - 0s 2ms/step

35/35 [==============================] - 0s 3ms/step

35/35 [==============================] - 0s 4ms/step

35/35 [==============================] - 0s 2ms/step

35/35 [==============================] - 0s 3ms/step

35/35 [==============================] - 0s 3ms/step

35/35 [==============================] - 0s 2ms/step

35/35 [==============================] - 0s 4ms/step

35/35 [==============================] - 0s 2ms/step

 33%|███▎      | 33/100 [04:11<10:41,  9.58s/trial, best loss: 0.004638240601792921]

35/35 [==============================] - 0s 2ms/step

35/35 [==============================] - 0s 3ms/step

35/35 [==============================] - 0s 3ms/step

35/35 [==============================] - 0s 2ms/step

35/35 [==============================] - 0s 2ms/step

35/35 [==============================] - 0s 3ms/step

35/35 [==============================] - 0s 3ms/step

35/35 [==============================] - 2s 2ms/step

35/35 [==============================] - 0s 2ms/step

35/35 [==============================] - 0s 2ms/step

35/35 [==============================] - 0s 4ms/step

35/35 [==============================] - 0s 3ms/step

35/35 [==============================] - 0s 5ms/step

35/35 [==============================] - 0s 3ms/step

35/35 [==============================] - 0s 3ms/step

35/35 [==============================] - 0s 2ms/step

35/35 [==============================] - 0s 3ms/step

35/35 [==============================] - 0s 2ms/step

35/35 [=====================

35/35 [==============================] - 0s 2ms/step

35/35 [==============================] - 0s 3ms/step

35/35 [==============================] - 0s 6ms/step

35/35 [==============================] - 0s 2ms/step

35/35 [==============================] - 0s 2ms/step

35/35 [==============================] - 0s 3ms/step

35/35 [==============================] - 0s 4ms/step

35/35 [==============================] - 0s 2ms/step

35/35 [==============================] - 0s 3ms/step

35/35 [==============================] - 0s 4ms/step

35/35 [==============================] - 0s 2ms/step

35/35 [==============================] - 0s 2ms/step

35/35 [==============================] - 0s 2ms/step

35/35 [==============================] - 0s 4ms/step

35/35 [==============================] - 0s 3ms/step

35/35 [==============================] - 0s 3ms/step

100%|██████████| 100/100 [13:17<00:00,  7.98s/trial, best loss: 0.004638240601792921]
Best parameters: {'activation_hidden_mse_4_layers'

In [ ]:
best

{'activation_hidden_mse_4_layers': 1,
 'batch_size_mse_4_layers': 450.0,
 'initialization_hidden_mse_4_layers': 0,
 'layer_architecture': 3,
 'loss_choice_4_layers': 0,
 'max_lr_mse_4_layers': 0.30774990591075846,
 'n_neurons_mse_4_layers': 240.0,
 'optimizer_mse_4_layers': 0,
 'start_lr_mse_4_layers': 0.09439376350813275}

In [ ]:
start_lr = 0.02298
max_lr = 1.526
batch_size = 450
N_EPOCHS = 100

tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
checkpoint_cb = keras.callbacks.ModelCheckpoint("Elections_2022.h5", save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=50, restore_best_weights=True)
onecycle_cb = OneCycleScheduler(N_EPOCHS*len(X_train)//batch_size, max_lr, start_rate=start_lr)

metrics = [tf.keras.metrics.MeanAbsoluteError()]

model = build_model(n_hidden=3, n_neurons=200, activation_list=['elu', 'elu', 'elu','sigmoid'], initialization_list=['lecun_normal', 'lecun_normal', 'lecun_normal'],
            learning_rate=1, loss='mse', input_shape=(81,), metrics=metrics, optimizer='nesterov')

model.fit(X_train, y_train, epochs=N_EPOCHS, validation_split=0.2, verbose=1, callbacks=[early_stopping_cb, onecycle_cb], batch_size=batch_size)

Epoch 1/100
7/8 [=========================>....] - ETA: 0s - loss: 0.0296 - mean_absolute_error: 0.1318

8/8 [==============================] - 2s 79ms/step - loss: 0.0287 - mean_absolute_error: 0.1298 - val_loss: 0.0196 - val_mean_absolute_error: 0.1068
Epoch 2/100
8/8 [==============================] - 0s 33ms/step - loss: 0.0133 - mean_absolute_error: 0.0882 - val_loss: 0.0106 - val_mean_absolute_error: 0.0807
Epoch 3/100
8/8 [==============================] - 0s 26ms/step - loss: 0.0086 - mean_absolute_error: 0.0728 - val_loss: 0.0085 - val_mean_absolute_error: 0.0712
Epoch 4/100
8/8 [==============================] - 0s 30ms/step - loss: 0.0069 - mean_absolute_error: 0.0653 - val_loss: 0.0069 - val_mean_absolute_error: 0.0649
Epoch 5/100
8/8 [==============================] - 0s 29ms/step - loss: 0.0060 - mean_absolute_error: 0.0603 - val_loss: 0.0063 - val_mean_absolute_error: 0.0615
Epoch 6/100
8/8 [==============================] - 0s 19ms/step - loss: 0.0054 - mean_absolute_error: 0.0573 - val_loss: 0.0058 - val_mean_absolute_error: 0.0584
Epoch 7/100
8/8 [=======================

In [ ]:
model.evaluate(X_test, y_test)

35/35 [==============================] - 0s 5ms/step - loss: 0.0016 - mean_absolute_error: 0.0544


[0.0016055339947342873, 0.05438436195254326]

## Test 2

Here we fix the number of layers and neurons and attempt to regularize to avoid overfitting ('stretch pants' strategy)

In [ ]:
from hyperopt import hp, fmin, tpe, Trials
from sklearn.metrics import mean_squared_error
import math

# Define the search space for hyperparameters
N_LAYERS = 5
N_NEURONS = 300

space = hp.choice('loss_choice'.format(n_layers),[
  {'n_hidden': N_LAYERS,
   'n_neurons': N_NEURONS,
  'activation_list': [hp.choice('activation_hidden_mse', ['selu', 'elu'])]*N_LAYERS + ['sigmoid'],
  'initialization_list': [hp.choice('initialization_hidden_mse', ['lecun_normal', 'he_normal'])]*N_LAYERS,
  #'n_neurons': hp.quniform('n_neurons_mse'.format(n_layers), 140, 300, 40),
  #'learning_rate': hp.loguniform('learning_rate_mse_{}_layers'.format(n_layers), -4, -2),
  'optimizer': hp.choice('optimizer_mse', ['momentum', 'nesterov', 'adam']),
  'start_learning_rate': hp.loguniform('start_lr_mse', math.log(1e-4), math.log(0.1)),
  'max_learning_rate': hp.loguniform('max_lr_mse', math.log(0.1), math.log(10)),
  'batch_size_0': hp.quniform('batch_size_mse', 32, 832, 50),
  'loss': 'mse'
  },
  {'n_hidden': N_LAYERS,
   'n_neurons': N_NEURONS,
  'activation_list': [hp.choice('activation_hidden_huber', ['selu', 'elu'])]*N_LAYERS + ['sigmoid'],
  'initialization_list': [hp.choice('initialization_hidden_huber', ['lecun_normal', 'he_normal'])]*N_LAYERS,
  #'n_neurons': hp.quniform('n_neurons_huber'.format(n_layers), 140, 300, 40),
  #'learning_rate': hp.loguniform('learning_rate_huber_{}_layers'.format(n_layers), -4, -2),
  'optimizer': hp.choice('optimizer_huber', ['momentum', 'nesterov', 'adam']),
  'start_learning_rate': hp.loguniform('start_lr_huber', math.log(1e-4), math.log(0.1)),
  'max_learning_rate': hp.loguniform('max_lr_huber', math.log(0.1), math.log(10)),
  'batch_size_0': hp.quniform('batch_size_huber', 32, 832, 50),
  'loss': 'huber',
  'delta': hp.uniform('delta', 0.04, 0.1)
  }])


# Objective function to minimize (or maximize)
def objective(params):
    N_EPOCHS = 100
    batch_size = int(params['batch_size_0'])
    model = build_model(**params, input_shape=(81,))

    # Assuming you have your data ready, otherwise replace X_train, y_train with your data
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

    max_learning_rate = params['max_learning_rate']
    start_learning_rate = params['start_learning_rate']


    onecycle_cb = OneCycleScheduler(N_EPOCHS*(len(X_train)//batch_size), max_learning_rate, start_rate=start_learning_rate)
    model.fit(X_train, y_train, epochs=N_EPOCHS, validation_split = 0.15, verbose=0, callbacks=[early_stopping_cb, onecycle_cb], batch_size=batch_size)

    # Evaluate model on validation data
    y_pred = model.predict(X_val)
    error = mean_squared_error(y_val, y_pred)
    return error


# Run optimization
trials = Trials()
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=100, trials=trials)

print("Best parameters:", best)
